In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
class Block1(nn.Module):
    def __init__(self, ch):
        super(Block1, self).__init__()
        self.bn = nn.BatchNorm2d(ch)
        self.conv = nn.Conv2d(ch, ch, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        shortcut = x
        out = self.bn(x)
        out = self.relu(out)
        out = self.conv(out)
        out = self.bn(out)
        out = self.relu(out)
        out = self.conv(out)
        out += shortcut
        return out

In [ ]:
class Block2(nn.Module):
    def __init__(self, ch):
        super(Block2, self).__init__()
        self.bn1 = nn.BatchNorm2d(ch)
        self.bn2 = nn.BatchNorm2d(2*ch)
        self.conv1 = nn.Conv2d(ch, 2*ch, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(2*ch, 2*ch, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv3 = nn.Conv2d(ch, 2*ch, kernel_size=1)
    
    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        shortcut = self.conv3(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out += shortcut
        return out

In [ ]:
class IdentityResNet(nn.Module):
    
    def __init__(self, nblk_stage1, nblk_stage2, nblk_stage3, nblk_stage4):
        super(IdentityResNet, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.stage1 = self.get_layers1(64, nblk_stage1)
        self.stage2 = self.get_layers2(64, nblk_stage2)
        self.stage3 = self.get_layers2(128, nblk_stage3)
        self.stage4 = self.get_layers2(256, nblk_stage4)
        self.avg_pool = nn.AvgPool2d(4, stride=4)
        self.fc_out = nn.Linear(512, 10)


    def get_layers1(self, ch, num_layers):
        layers_list = nn.ModuleList([Block1(ch)])
        for _ in range(num_layers - 1):
            layers_list.append(Block1(ch))
        return nn.Sequential(*layers_list)


    def get_layers2(self, ch, num_layers):
        layers_list = nn.ModuleList([Block2(ch)])
        for _ in range(num_layers - 1):
            layers_list.append(Block1(ch))
        return nn.Sequential(*layers_list)
    
    
    def forward(self, x):
        x = self.conv0(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.avg_pool(x)
        x = self.fc_out(x)
        return x

In [ ]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('current device: ', dev)

current device:  cuda:0


In [ ]:

batch_size = 4

# preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load training data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

# load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# define network
net = IdentityResNet(nblk_stage1=2, nblk_stage2=2,
                     nblk_stage3=2, nblk_stage4=2)


model = net.to(dev)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
t_start = time.time()

In [ ]:
for epoch in range(5): 
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(dev), data[1].to(dev)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            t_end = time.time()
            print('elapsed:', t_end - t_start, ' sec')
            t_start = t_end

print('Finished Training')

RuntimeError: ignored

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with ...
    for data in testloader:
        images, labels = data[0].to(dev), data[1].to(dev)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# per-class accuracy
for i in range(10):
    print('Accuracy of %5s' % (classes[i]), ': ',
          100 * class_correct[i] / class_total[i], '%')

# overall accuracy
print('Overall Accurracy: ', (sum(class_correct) / sum(class_total)) * 100, '%')